<a href="https://colab.research.google.com/github/ashutosh3060/recipe-sales-forecast/blob/main/Data_Transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Table of Contents

0. Libraries
1. User-Defined Functions
2. Import Data and Basic Understanding
3. Data Quality Checks   
  3.1Duplicate    
        3.1.1 Complete row duplicate
        3.1.2 Unique Id duplicate
  3.2 Missing value
4. Feature Engineering   
  4.1 Independent Feature Creation   
  4.2 Missing Value Imputation      
5. Save Final Dataset having new features
6. Ideas for Future work

## 0. Libraries 

In [1]:
# warnings
import warnings
warnings.filterwarnings("ignore")

# Numpy, Pandas
import numpy as np
import pandas as pd

# Display settings
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 15)

## 1. User-defined Functions

In [2]:
def null_perc_check (df):
    '''
    Calculates missing value count and percentage for all the columns in a dataframe

    Inputs
    -------
    df : dataframe
        The dataframe for which missing value distribution needs to checked

    Output
    -------
    dataframe
        a dataframe showing missing value count and percentage for all the columns
    '''
    missing_value_df = pd.DataFrame(index = df.keys(), data =df.isnull().sum(), columns = ['Missing_Value_Count'])
    missing_value_df['Missing_Value_Percentage'] = np.round(((df.isnull().mean())*100),1)
    sorted_df = missing_value_df.sort_values('Missing_Value_Count',ascending= False)
    return sorted_df

def order_dist_perc_calc(df, unique_id, feature):   
    '''
    Calculates the order % for unique categories from the given independent variable
    
    Inputs
    ----------
    df : dataframe
        The raw dataframe which is used for new feature creation
    unique_id : string
        Unique id name
    feature : string
        Name of the feature for which order % of unique categories need to be calculated

    Output
    -------
    dataframe
        a dataframe showing order percentage of unique categories for each customer
    '''
    feature_grp = df.groupby([unique_id, feature]).agg({feature: 'count'})
    feature_pct = feature_grp.groupby(level=0).apply(lambda x: round(100 * x / float(x.sum()),2))
    feature_pct.rename(columns={feature: f"{feature}_pct"}, inplace=True)
    feature_id = feature_pct.reset_index()
    df_feature_id = pd.DataFrame(feature_id.pivot(index=unique_id, columns=feature, values=f'{feature}_pct'))
    df_feature_id.reset_index()
    df_feature_id = df_feature_id.fillna(0)
    df_feature_id = df_feature_id.apply(pd.to_numeric)
    df_feature_id = df_feature_id.add_prefix(f'{feature}_')
    return df_feature_id

## 2. Import Data and Basic Understanding

In [3]:
# Define the path, filename

train_path = "/content/"
train_file = "train_data_cleaned.csv"
test_path = "/content/"
test_file = "test_data_cleaned.csv"

In [4]:
# import the train dataset as a dataframe

df_train = pd.read_csv(train_path+train_file)
df_test = pd.read_csv(test_path+test_file)

In [5]:
# df_train: shape and first few records

print(df_train.shape)
df_train.head(3)

(2939, 27)


,year_week,recipe_id,recipe_name,product_type,calories,carbs,cooking_time,cuisine,description,difficulty,dish_type,heat_level,fat,is_classic,number_of_ingredients_per_recipe,preferences,carbs_content,dish_types,seasons,protein_types,proteins,course_type,cooking_tip,meta_tags,percentage_of_local_ingredients,protein_cuts,sales
0,201801,14919,Shrimp & Br...,2 person,687.0,99.2,time_level_4,asian,For this di...,easy,fish,optional_heat,18.6,No,7,shellfish,rice,fish_and_si...,all_seasons,shellfish,29.5,main,NaN,healthy_choice,0.0,NaN,3567
1,201801,14920,Caramelized...,2 person,779.0,81.3,time_level_4,new_american,Red lentil ...,easy,veggie,no_heat,38.5,No,5,no_protein,pasta_incl_...,pasta,all_seasons,no_protein,30.0,main,NaN,quick,0.0,no_cut,2532
2,201801,14449,AFC Vegetar...,z_amazon,642.0,92.8,NaN,NaN,Pad Thai is...,easy,veggie,NaN,25.6,No,10,NaN,NaN,NaN,NaN,NaN,16.2,NaN,NaN,NaN,NaN,NaN,6


In [6]:
# df_test: shape and first few records

print(df_test.shape)
df_test.head(3)

(320, 27)


,year_week,recipe_id,recipe_name,product_type,calories,carbs,cooking_time,cuisine,description,difficulty,dish_type,heat_level,fat,is_classic,number_of_ingredients_per_recipe,preferences,carbs_content,dish_types,seasons,protein_types,proteins,course_type,cooking_tip,meta_tags,percentage_of_local_ingredients,protein_cuts,sales
0,201945,38287,Juicy Lucy ...,2 person,990.0,65.0,time_level_5,mexican,Just when y...,easy,meat,no_heat,64.0,No,9,"beef,dairy","potato,pank...",meat_and_side,NaN,"beef,dairy",42.0,main,NaN,kid_friendly,0.0,ground_mince,NaN
1,201945,38309,Baja-Style ...,2 person,790.0,60.0,time_level_4,mexican,Pico de gal...,easy,veggie,no_heat,58.0,No,9,dairy,tortillas_a...,tacos_torti...,all_seasons,dairy,12.0,main,NaN,quick,0.0,no_cut,NaN
2,201945,38310,Baja-Style ...,family,690.0,60.0,time_level_4,mexican,Pico de gal...,easy,veggie,no_heat,46.0,No,9,dairy,tortillas_a...,tacos_torti...,all_seasons,dairy,12.0,main,NaN,quick,0.0,no_cut,NaN


In [7]:
# Number of unique recipe in both the datasets

print(f'df_train: {df_train.recipe_id.nunique()}')
print(f'df_test: {df_test.recipe_id.nunique()}')

df_train: 1931
df_test: 318


* As mentioned in the problem document, it seems like multiple records are present for a single recipe(There are chances of same recipe in different weeks)

* Need to check if same recipe is repeated in a single week

In [8]:
# Datatypes overview of the train dataset

df = df_train.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2939 entries, 0 to 2938
Data columns (total 27 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   year_week                         2939 non-null   int64  
 1   recipe_id                         2939 non-null   int64  
 2   recipe_name                       2939 non-null   object 
 3   product_type                      2939 non-null   object 
 4   calories                          2933 non-null   float64
 5   carbs                             2892 non-null   float64
 6   cooking_time                      2648 non-null   object 
 7   cuisine                           2106 non-null   object 
 8   description                       2935 non-null   object 
 9   difficulty                        2930 non-null   object 
 10  dish_type                         2521 non-null   object 
 11  heat_level                        2223 non-null   object 
 12  fat   

* Few columns are having missing values.
* Proper analysis of features is required in Exploratory Data Analysis section.

In [9]:
# Datatypes overview of the test dataset

df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 27 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   year_week                         320 non-null    int64  
 1   recipe_id                         320 non-null    int64  
 2   recipe_name                       320 non-null    object 
 3   product_type                      320 non-null    object 
 4   calories                          320 non-null    float64
 5   carbs                             320 non-null    float64
 6   cooking_time                      318 non-null    object 
 7   cuisine                           318 non-null    object 
 8   description                       320 non-null    object 
 9   difficulty                        320 non-null    object 
 10  dish_type                         320 non-null    object 
 11  heat_level                        306 non-null    object 
 12  fat     

* Here also, missing values are present.

## 3. Data Quality Checks

### 3.1 Duplicate record checks

#### 3.1.1 Complete Row Duplicate

In [10]:
# Check duplicates

check_dup = df.duplicated(subset=None, keep='first')
print(f'Number of Total records:  {df.shape[0]}')
print(f'Number of Unique records: {check_dup.shape[0]}')
print(f'Number of Duplicates: {df.shape[0] - check_dup.shape[0]}')

Number of Total records:  2939
Number of Unique records: 2939
Number of Duplicates: 0


* Both the total and unique records are same, so there are no duplicate present.

#### 3.1.2 recipe_id (unique identifier) Duplicate

In [11]:
# Group by week, recipe and check if count of records is more than 1

# train dataset
df_recipe_dup_train = df[df.groupby(['year_week','recipe_id'])['recipe_name'].transform('count') > 1]
print(f'Number of Duplicates in train dataset: {df_recipe_dup_train.shape[0]}')

# test dataset
df_recipe_dup_test = df_test[df_test.groupby(['year_week','recipe_id'])['recipe_name'].transform('count') > 1]
print(f'Number of Duplicates in test dataset: {df_recipe_dup_test.shape[0]}')

Number of Duplicates in train dataset: 0
Number of Duplicates in test dataset: 0


* There are no such recipe duplicated in any of the week

### 3.2 Null check - Missing value percentage

In [12]:
# Missing value count and percentage check for train dataset

null_perc_check(df)

,Missing_Value_Count,Missing_Value_Percentage
cooking_tip,2939,100.0
protein_cuts,991,33.7
seasons,925,31.5
meta_tags,870,29.6
cuisine,833,28.3
heat_level,716,24.4
dish_types,704,24.0
course_type,703,23.9
protein_types,701,23.9
carbs_content,701,23.9


* Many columns have missing values.
* **cooking_tip** is a complete null column.
* For all other columns, proper analysis of outlier is required before setting the method and threshold for imputation.

In [13]:
# Check the missing value percentage for test dataset

null_perc_check(df_test)

,Missing_Value_Count,Missing_Value_Percentage
sales,320,100.0
cooking_tip,320,100.0
seasons,226,70.6
protein_cuts,22,6.9
heat_level,14,4.4
meta_tags,12,3.8
dish_types,4,1.2
cooking_time,2,0.6
cuisine,2,0.6
course_type,2,0.6


* In Test dataset also, many columns are having nulls.
* **Sales** being the traget column makes sense. **cooking_tip** is complete null as train dataset.
* All other missing values columns need analysis before imputation

## 4. Feature Engineering

### 4.1 Feature Transformation

#### 4.1.1 Columns to be dropped
* Below columns need to be dropped due to the presence of all same values  
i) difficulty     
ii) is_classic    
iii) course_type  
*  Below needs to dropped due to high % of missing values   
i) cooking_tip

#### 4.1.2 Categorical Encoding  
i) product_type   
ii) cooking_time   
iii) cuisine   
iv) dish_type   
v) heat_level       
vi) protein_cuts

#### 4.1.3 New Feature Extraction
i) week number from year_week variable  
* Create new features for presence of each unique values in the below variables  
i) preferences  
ii) carbs_content   
iii) dish_types   
iv) protein_types   
v) meta_tags  
vi) seasons 

#### 4.1.1 Drop Columns

In [18]:
# Train Data
print(df.shape)
df = df.drop(["difficulty", "is_classic", "course_type", "cooking_tip"], axis = 1) 
print(df.shape)

(2939, 27)
(2939, 23)


In [19]:
# Test Data
print(df_test.shape)
df_test = df_test.drop(["difficulty", "is_classic", "course_type", "cooking_tip"], axis = 1) 
print(df_test.shape)

(320, 27)
(320, 23)


#### 4.1.2 Categorical Encoding

i) product_type  
ii) cooking_time  
iii) cuisine  
iv) dish_type   
v) heat_level    
vi) protein_cuts  
vii) preferences  
viii) carbs_content  
ix) dish_types  
x) seasons  
xi) protein_types     
xii) meta_tags   

In [20]:
# Data type casting for all the categorical variables

df["product_type_cat"] = df["product_type"].astype('category')
df["cooking_time_cat"] = df["cooking_time"].astype('category')
df["cuisine_cat"] = df["cuisine"].astype('category')
df["dish_type_cat"] = df["dish_type"].astype('category')
df["heat_level_cat"] = df["heat_level"].astype('category')
df["protein_cuts_cat"] = df["protein_cuts"].astype('category')
df["preferences_cat"] = df["preferences"].astype('category')
df["carbs_content_cat"] = df["carbs_content"].astype('category')
df["dish_types_cat"] = df["dish_types"].astype('category')
df["seasons_cat"] = df["seasons"].astype('category')
df["protein_types_cat"] = df["protein_types"].astype('category')
df["meta_tags_cat"] = df["meta_tags"].astype('category')

In [21]:
# Assign a class to all the values and null will be -1 

df["product_type_cat"] = df["product_type_cat"].cat.codes
df["cooking_time_cat"] = df["cooking_time_cat"].cat.codes
df["cuisine_cat"] = df["cuisine_cat"].cat.codes
df["dish_type_cat"] = df["dish_type_cat"].cat.codes
df["heat_level_cat"] = df["heat_level_cat"].cat.codes
df["protein_cuts_cat"] = df["protein_cuts_cat"].cat.codes
df["preferences_cat"] = df["preferences_cat"].cat.codes
df["carbs_content_cat"] = df["carbs_content_cat"].cat.codes
df["dish_types_cat"] = df["dish_types_cat"].cat.codes
df["seasons_cat"] = df["seasons_cat"].cat.codes
df["protein_types_cat"] = df["protein_types_cat"].cat.codes
df["meta_tags_cat"] = df["meta_tags_cat"].cat.codes
print(df.shape)
df.head()

(2939, 35)


,year_week,recipe_id,recipe_name,product_type,calories,carbs,cooking_time,cuisine,description,dish_type,heat_level,fat,number_of_ingredients_per_recipe,preferences,carbs_content,dish_types,seasons,protein_types,proteins,meta_tags,percentage_of_local_ingredients,protein_cuts,sales,product_type_cat,cooking_time_cat,cuisine_cat,dish_type_cat,heat_level_cat,protein_cuts_cat,preferences_cat,carbs_content_cat,dish_types_cat,seasons_cat,protein_types_cat,meta_tags_cat
0,201801,14919,Shrimp & Br...,2 person,687.0,99.2,time_level_4,asian,For this di...,fish,optional_heat,18.6,7,shellfish,rice,fish_and_si...,all_seasons,shellfish,29.5,healthy_choice,0.0,NaN,3567,0,1,1,0,3,-1,39,31,11,0,32,0
1,201801,14920,Caramelized...,2 person,779.0,81.3,time_level_4,new_american,Red lentil ...,veggie,no_heat,38.5,5,no_protein,pasta_incl_...,pasta,all_seasons,no_protein,30.0,quick,0.0,no_cut,2532,0,1,20,2,1,8,29,24,49,0,24,10
2,201801,14449,AFC Vegetar...,z_amazon,642.0,92.8,NaN,NaN,Pad Thai is...,veggie,NaN,25.6,10,NaN,NaN,NaN,NaN,NaN,16.2,NaN,NaN,NaN,6,3,-1,-1,2,-1,-1,-1,-1,-1,-1,-1,-1
3,201801,11963,Shrimp & Br...,family,620.0,93.2,time_level_4,asian,For this di...,fish,optional_heat,13.7,7,shellfish,rice,fish_and_si...,all_seasons,shellfish,28.7,healthy_choice,0.0,NaN,642,2,1,1,0,3,-1,39,31,11,0,32,0
4,201801,14914,Chicken & P...,family,610.0,67.0,time_level_4,new_american,We've never...,meat,no_heat,20.0,7,poultry,sweet_potat...,soup,all_seasons,poultry,36.0,healthy_choice,0.0,breast,1492,2,1,20,1,1,0,36,39,84,0,30,0


In [22]:
# Test Data

# Data type casting for all the categorical variables
df_test["product_type_cat"] = df_test["product_type"].astype('category')
df_test["cooking_time_cat"] = df_test["cooking_time"].astype('category')
df_test["cuisine_cat"] = df_test["cuisine"].astype('category')
df_test["dish_type_cat"] = df_test["dish_type"].astype('category')
df_test["heat_level_cat"] = df_test["heat_level"].astype('category')
df_test["protein_cuts_cat"] = df_test["protein_cuts"].astype('category')
df_test["preferences_cat"] = df_test["preferences"].astype('category')
df_test["carbs_content_cat"] = df_test["carbs_content"].astype('category')
df_test["dish_types_cat"] = df_test["dish_types"].astype('category')
df_test["seasons_cat"] = df_test["seasons"].astype('category')
df_test["protein_types_cat"] = df_test["protein_types"].astype('category')
df_test["meta_tags_cat"] = df_test["meta_tags"].astype('category')

# Assign a class to all the values and null will be -1 
df_test["product_type_cat"] = df_test["product_type_cat"].cat.codes
df_test["cooking_time_cat"] = df_test["cooking_time_cat"].cat.codes
df_test["cuisine_cat"] = df_test["cuisine_cat"].cat.codes
df_test["dish_type_cat"] = df_test["dish_type_cat"].cat.codes
df_test["heat_level_cat"] = df_test["heat_level_cat"].cat.codes
df_test["protein_cuts_cat"] = df_test["protein_cuts_cat"].cat.codes
df_test["preferences_cat"] = df_test["preferences_cat"].cat.codes
df_test["carbs_content_cat"] = df_test["carbs_content_cat"].cat.codes
df_test["dish_types_cat"] = df_test["dish_types_cat"].cat.codes
df_test["seasons_cat"] = df_test["seasons_cat"].cat.codes
df_test["protein_types_cat"] = df_test["protein_types_cat"].cat.codes
df_test["meta_tags_cat"] = df_test["meta_tags_cat"].cat.codes

#### 4.1.3 New Feature Extraction

* week number from year_week variable

In [23]:
# Train Data
# Create weeknum variable

df["week_num"] = df.year_week.astype(str)
df["week_num"] = df["week_num"].str[4:]
df["week_num"] = df["week_num"].astype(int)
df["week_num"].unique()


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52])

In [24]:
df.head()

,year_week,recipe_id,recipe_name,product_type,calories,carbs,cooking_time,cuisine,description,dish_type,heat_level,fat,number_of_ingredients_per_recipe,preferences,carbs_content,dish_types,seasons,protein_types,proteins,meta_tags,percentage_of_local_ingredients,protein_cuts,sales,product_type_cat,cooking_time_cat,cuisine_cat,dish_type_cat,heat_level_cat,protein_cuts_cat,preferences_cat,carbs_content_cat,dish_types_cat,seasons_cat,protein_types_cat,meta_tags_cat,week_num
0,201801,14919,Shrimp & Br...,2 person,687.0,99.2,time_level_4,asian,For this di...,fish,optional_heat,18.6,7,shellfish,rice,fish_and_si...,all_seasons,shellfish,29.5,healthy_choice,0.0,NaN,3567,0,1,1,0,3,-1,39,31,11,0,32,0,1
1,201801,14920,Caramelized...,2 person,779.0,81.3,time_level_4,new_american,Red lentil ...,veggie,no_heat,38.5,5,no_protein,pasta_incl_...,pasta,all_seasons,no_protein,30.0,quick,0.0,no_cut,2532,0,1,20,2,1,8,29,24,49,0,24,10,1
2,201801,14449,AFC Vegetar...,z_amazon,642.0,92.8,NaN,NaN,Pad Thai is...,veggie,NaN,25.6,10,NaN,NaN,NaN,NaN,NaN,16.2,NaN,NaN,NaN,6,3,-1,-1,2,-1,-1,-1,-1,-1,-1,-1,-1,1
3,201801,11963,Shrimp & Br...,family,620.0,93.2,time_level_4,asian,For this di...,fish,optional_heat,13.7,7,shellfish,rice,fish_and_si...,all_seasons,shellfish,28.7,healthy_choice,0.0,NaN,642,2,1,1,0,3,-1,39,31,11,0,32,0,1
4,201801,14914,Chicken & P...,family,610.0,67.0,time_level_4,new_american,We've never...,meat,no_heat,20.0,7,poultry,sweet_potat...,soup,all_seasons,poultry,36.0,healthy_choice,0.0,breast,1492,2,1,20,1,1,0,36,39,84,0,30,0,1


In [25]:
# Test Data
df_test["week_num"] = df_test.year_week.astype(str)
df_test["week_num"] = df_test["week_num"].str[4:]
df_test["week_num"] = df_test["week_num"].astype(int)

In [26]:
df.columns

Index(['year_week', 'recipe_id', 'recipe_name', 'product_type', 'calories',
       'carbs', 'cooking_time', 'cuisine', 'description', 'dish_type',
       'heat_level', 'fat', 'number_of_ingredients_per_recipe', 'preferences',
       'carbs_content', 'dish_types', 'seasons', 'protein_types', 'proteins',
       'meta_tags', 'percentage_of_local_ingredients', 'protein_cuts', 'sales',
       'product_type_cat', 'cooking_time_cat', 'cuisine_cat', 'dish_type_cat',
       'heat_level_cat', 'protein_cuts_cat', 'preferences_cat',
       'carbs_content_cat', 'dish_types_cat', 'seasons_cat',
       'protein_types_cat', 'meta_tags_cat', 'week_num'],
      dtype='object')

In [27]:
# Training Data
# Select the order of feature for better readability and ease of analysis

df_new_ftr_train = df[['year_week', 'week_num', 'recipe_id', 'recipe_name', 'calories', 'carbs', 
                 'fat', 'number_of_ingredients_per_recipe', 
                 'product_type_cat', 'cooking_time_cat', 'cuisine_cat', 'dish_type_cat', 
                 'heat_level_cat', 'protein_cuts_cat', 'preferences_cat', 
                 'carbs_content_cat', 'dish_types_cat', 'seasons_cat', 
                 'protein_types_cat', 'meta_tags_cat', 'sales']].copy()
df_new_ftr_train.index = np.arange(1, len(df_new_ftr_train)+1)
print(df_new_ftr_train.shape)
df_new_ftr_train.tail(2)

(2939, 21)


,year_week,week_num,recipe_id,recipe_name,calories,carbs,fat,number_of_ingredients_per_recipe,product_type_cat,cooking_time_cat,cuisine_cat,dish_type_cat,heat_level_cat,protein_cuts_cat,preferences_cat,carbs_content_cat,dish_types_cat,seasons_cat,protein_types_cat,meta_tags_cat,sales
2938,201944,44,38199,Smothered P...,760.0,51.0,42.0,8,0,2,20,1,1,3,34,14,14,0,29,5,1134
2939,201944,44,38284,Lemon-Chick...,920.0,52.0,47.0,6,2,2,20,1,1,-1,36,27,18,-1,30,1,190


In [28]:
# Test Data
# Select the order of feature for better readability and ease of analysis

df_new_ftr_test = df_test[['year_week', 'week_num', 'recipe_id', 'recipe_name', 'calories', 'carbs', 
                 'fat', 'number_of_ingredients_per_recipe', 
                 'product_type_cat', 'cooking_time_cat', 'cuisine_cat', 'dish_type_cat', 
                 'heat_level_cat', 'protein_cuts_cat', 'preferences_cat', 
                 'carbs_content_cat', 'dish_types_cat', 'seasons_cat', 
                 'protein_types_cat', 'meta_tags_cat', 'sales']].copy()
df_new_ftr_test.index = np.arange(1, len(df_new_ftr_test)+1)
print(df_new_ftr_test.shape)
df_new_ftr_test.tail(2)

(320, 21)


,year_week,week_num,recipe_id,recipe_name,calories,carbs,fat,number_of_ingredients_per_recipe,product_type_cat,cooking_time_cat,cuisine_cat,dish_type_cat,heat_level_cat,protein_cuts_cat,preferences_cat,carbs_content_cat,dish_types_cat,seasons_cat,protein_types_cat,meta_tags_cat,sales
319,201952,52,40226,Quick & Eas...,1170.0,80.0,72.0,9,0,2,13,1,1,5,3,8,29,-1,2,4,NaN
320,201952,52,40393,Moroccan Ro...,700.0,55.0,39.0,8,1,3,13,1,1,10,7,14,6,-1,5,0,NaN


* In future, for below columns, extract the unique values and create separate features out of these. Values will be 1 if the unique value is present else 0.

i) preferences  
ii) carbs_content  
iii) dish_types  
iv) seasons  
v) protein_types     
vi) meta_tags   


### 4.2 Missing Value Imputation

In [34]:
df_new_ftr_train = df_new_ftr_train.fillna(df_new_ftr_train.mean())

## 5. Save final dataset having new features

In [35]:
# Export the dataframe to a csv
df_new_ftr_train.to_csv('train_data_transformed.csv', index=False)
df_new_ftr_test.to_csv('test_data_transformed.csv', index=False)

In [36]:
# Cross-check the data in csv file for Train Data

df_train_data = pd.read_csv('train_data_transformed.csv')
print(df_train_data.shape)
print('--'*20)
print('--'*20)
print(df_train_data.columns)
print('--'*20)
print('--'*20)
df_train_data.head(3)

(2939, 21)
----------------------------------------
----------------------------------------
Index(['year_week', 'week_num', 'recipe_id', 'recipe_name', 'calories',
       'carbs', 'fat', 'number_of_ingredients_per_recipe', 'product_type_cat',
       'cooking_time_cat', 'cuisine_cat', 'dish_type_cat', 'heat_level_cat',
       'protein_cuts_cat', 'preferences_cat', 'carbs_content_cat',
       'dish_types_cat', 'seasons_cat', 'protein_types_cat', 'meta_tags_cat',
       'sales'],
      dtype='object')
----------------------------------------
----------------------------------------


,year_week,week_num,recipe_id,recipe_name,calories,carbs,fat,number_of_ingredients_per_recipe,product_type_cat,cooking_time_cat,cuisine_cat,dish_type_cat,heat_level_cat,protein_cuts_cat,preferences_cat,carbs_content_cat,dish_types_cat,seasons_cat,protein_types_cat,meta_tags_cat,sales
0,201801,1,14919,Shrimp & Br...,687.0,99.2,18.6,7,0,1,1,0,3,-1,39,31,11,0,32,0,3567
1,201801,1,14920,Caramelized...,779.0,81.3,38.5,5,0,1,20,2,1,8,29,24,49,0,24,10,2532
2,201801,1,14449,AFC Vegetar...,642.0,92.8,25.6,10,3,-1,-1,2,-1,-1,-1,-1,-1,-1,-1,-1,6


In [37]:
# Cross-check the data in csv file for Test Data

df_test_data = pd.read_csv('test_data_transformed.csv')
print(df_test_data.shape)
print('--'*20)
print('--'*20)
print(df_test_data.columns)
print('--'*20)
print('--'*20)
df_test_data.head(3)

(320, 21)
----------------------------------------
----------------------------------------
Index(['year_week', 'week_num', 'recipe_id', 'recipe_name', 'calories',
       'carbs', 'fat', 'number_of_ingredients_per_recipe', 'product_type_cat',
       'cooking_time_cat', 'cuisine_cat', 'dish_type_cat', 'heat_level_cat',
       'protein_cuts_cat', 'preferences_cat', 'carbs_content_cat',
       'dish_types_cat', 'seasons_cat', 'protein_types_cat', 'meta_tags_cat',
       'sales'],
      dtype='object')
----------------------------------------
----------------------------------------


,year_week,week_num,recipe_id,recipe_name,calories,carbs,fat,number_of_ingredients_per_recipe,product_type_cat,cooking_time_cat,cuisine_cat,dish_type_cat,heat_level_cat,protein_cuts_cat,preferences_cat,carbs_content_cat,dish_types_cat,seasons_cat,protein_types_cat,meta_tags_cat,sales
0,201945,45,38287,Juicy Lucy ...,990.0,65.0,64.0,9,0,3,14,1,1,5,4,11,6,-1,3,4,NaN
1,201945,45,38309,Baja-Style ...,790.0,60.0,58.0,9,0,2,14,2,1,6,6,16,55,0,4,8,NaN
2,201945,45,38310,Baja-Style ...,690.0,60.0,46.0,9,1,2,14,2,1,6,6,16,55,0,4,8,NaN


* Looks good

## 6. Ideas for Future work

1. Data Transformation   

Create individual features for each unique value in the column for the below ones.  
 
i) preferences  
ii) carbs_content  
iii) dish_types  
iv) seasons  
v) protein_types     
vi) meta_tags 

********************************

********************************